In [ ]:
%%capture
!pip install gdown

In [ ]:
%%capture
import gdown
URL = 'https://drive.google.com/file/d/1GBBkwDzJ7HI1b-i_d9kpn-VWpaiHxRUj/view?usp=sharing'
gdown.download(URL, 'gnss_log.txt', quiet=True, fuzzy=True)

In [ ]:
import pandas as pd

column_names = [
    "Status",
    "UnixTimeMillis",
    "SignalCount",
    "SignalIndex",
    "ConstellationType",
    "Svid",
    "CarrierFrequencyHz",
    "Cn0DbHz",
    "AzimuthDegrees",
    "ElevationDegrees",
    "UsedInFix",
    "HasAlmanacData",
    "HasEphemerisData",
    "BasebandCn0DbHz"
]

df = pd.read_csv(
    'gnss_log.txt',
    comment="#",
    names=column_names,
    on_bad_lines='skip',
)
df.head(3)

In [ ]:
df['Svid'].nunique()

In [ ]:
def get_constellation_name(constellation_type):
    constellation_map = {
        1: "GPS",
        3: "Glonass",
        4: "QZSS",
        5: "BeiDou",
        6: "Galileo",
    }
    return constellation_map.get(constellation_type, "UNKNOWN")

df["satellite_type"] = df["ConstellationType"].apply(get_constellation_name)
df.head(3)

In [ ]:
df['BasebandCn0DbHz'].describe()

In [ ]:
df = df[df['BasebandCn0DbHz'] > 25]
df['Svid'].nunique()

In [ ]:
min_unix_time = df['UnixTimeMillis'].min(skipna=True)
max_unix_time = df['UnixTimeMillis'].max(skipna=True)

min_datetime_utc = pd.to_datetime(min_unix_time, unit='ms', utc=True)
max_datetime_utc = pd.to_datetime(max_unix_time, unit='ms', utc=True)

min_datetime_local = min_datetime_utc.tz_convert('Asia/Ho_Chi_Minh')
max_datetime_local = max_datetime_utc.tz_convert('Asia/Ho_Chi_Minh')

print("Start:", min_datetime_local)
print("End:", max_datetime_local)

In [ ]:
%%capture

import gdown

URL = 'https://drive.google.com/file/d/13R-tVolpJrSBPHcqLKms-9dGeL2nHMm3/view?usp=share_link'
gdown.download(URL, 'satellites.csv', quiet=True, fuzzy=True)

In [ ]:
import pandas as pd

satellites_df = pd.read_csv('satellites.csv')
satellites_df.head(3)

In [ ]:
from scipy.spatial import cKDTree

constellation_satellites = satellites_df[satellites_df['satellite_type'] == 'GPS']
observed_satellites = df[df['satellite_type'] == 'GPS']

tree = cKDTree(constellation_satellites[['altitude', 'azimuth']].values)
distances, indices = tree.query(observed_satellites[['ElevationDegrees', 'AzimuthDegrees']].values)

observed_satellites['norad_cat_id'] = constellation_satellites.iloc[indices]['satellite_id'].values
observed_satellites['satellite_altitude'] = constellation_satellites.iloc[indices]['altitude'].values
observed_satellites['satellite_azimuth'] = constellation_satellites.iloc[indices]['azimuth'].values

observed_satellites['altitude_delta'] = observed_satellites['ElevationDegrees'] - observed_satellites['satellite_altitude']
observed_satellites['azimuth_delta'] = observed_satellites['AzimuthDegrees'] - observed_satellites['satellite_azimuth']

observed_satellites.head()

# df.drop_duplicates(subset=['constellation', 'svid'])[['constellation', 'svid', 'norad_cat_id']].to_csv('svid_to_norad_id.csv', index=False)

In [ ]:
observed_satellites['norad_cat_id'].nunique()

In [ ]:
observed_satellites['Svid'].nunique()

In [ ]:
observed_satellites.groupby(['norad_cat_id', 'Svid']).agg(
    count=('BasebandCn0DbHz', 'size'),  # Count of rows for each unique combination
    max_cn0=('BasebandCn0DbHz', 'max')   # Maximum BasebandCn0DbHz for each group
)